In [289]:
import numpy as np
import pandas as pd
import itertools
from shapely.geometry import Polygon

In [290]:
# to_gk git
# to_1992 git
# to_2000 git
# pola git
# 

## Funkcje

In [291]:
def to_gk(fi, lam, lam_zero, a=6378137, e2=0.00669437999013):
    """przelicza współrzędne fi, lambda danej elipsoidy na x,y Gaussa-Krugera (w metrach), lam_zero to południk osiowy.
    fi, lam, lam_zero podajemy w stopniach."""
    a2 = a*a
    b2 = a2*(1 - e2)
    eprim2 = (a2 - b2) / b2 
    fi = np.deg2rad(fi)
    lam = np.deg2rad(lam)
    lam_zero = np.deg2rad(lam_zero)
    t = np.tan(fi)
    eta2 = eprim2 * (np.cos(fi))**2
    l = lam - lam_zero
    N = a / np.sqrt(1 - e2 * np.sin(fi) ** 2)
    A0 = 1 - (e2/4) - (3*(e2**2)/64) - (5*(e2**3)/256)
    A2 = (3/8) * (e2 + ((e2**2)/4) + (15*(e2**3))/128)
    A4 = (15/256) * (e2**2 + 3*(e2**3)/4)
    A6 = 35*(e2**3)/3072
    sigma = a * ( A0*fi - A2*np.sin(2*fi) + A4*np.sin(4*fi) - A6*np.sin(6*fi) )
    xgk = sigma + ((l**2)/2)*N*np.sin(fi)*np.cos(fi)* ( 1 + ((l**2)/12)*(np.cos(fi)**2)*(5-t**2+9*eta2+4*(eta2**2)) + ((l**4)/360)*(np.cos(fi)**4)*(61-58*t**2+t**4+270*eta2-330*eta2*t**2) )
    ygk = l*N*np.cos(fi)* ( 1 + ((l**2)/6)*(np.cos(fi)**2)*(1-t**2+eta2) + ((l**4)/120)*(np.cos(fi)**4)*(5-18*t**2+t**4+14*eta2-58*eta2*t**2) )
    return xgk, ygk

In [292]:
def to_1992(fi, lam):
    """przelicza fi, lam w stopniach na układ 1992 (x,y w metrach). Końcowe współrzędne mają 6 cyfr w 1992."""
    # m0_92 - skala długości na południku osiowym 19 stopni
    m0_92 = 0.9993
    xgk, ygk = to_gk(fi, lam, 19)
    # cechowanie współrzędnych
    x92 = m0_92*xgk - 5_300_000
    y92 = m0_92*ygk + 500_000
    return x92, y92

In [293]:
def to_2000(fi, lam):
    """przelicza fi, lam w stopniach na układ 2000 (x,y w metrach). Końcowe współrzędne mają 7 cyfr w 2000.
    Są cztery strefy odwzorowawcze. południki osiowe 15, 18, 21, 24. Pasy oznaczone numerami: 5, 6, 7, 8.
    m0 południka osiowego każdego pasa wynosi 0.999923."""
    if abs(lam-15) <= 1.5:
        osiowy = 15
    elif abs(lam-18) <= 1.5:
        osiowy = 18
    elif abs(lam-21) <= 1.5:
        osiowy = 21
    elif abs(lam-24) <= 1.5:
        osiowy = 24
        
    m0_2000 = 0.999923
    nr_strefy = osiowy/3
    xgk, ygk = to_gk(fi, lam, osiowy)
    x2000 = m0_2000 * xgk
    y2000 = m0_2000 * ygk + nr_strefy*1_000_000 + 500_000
    # f"{round(x2000, 3):.3f}"
    return x2000, y2000

In [294]:
def pole_czworokata(fi1, lam1, fi2, lam2, e2=0.00669437999013, a=6378137):  # , b=6356752.314140347
    """podajemy dwa narożniki siatki, dostajemy pole w metrach. pole na elipsoidzie."""
    b = a * (1 - e2)**0.5
    e = np.sqrt(e2)
    lam1 = np.deg2rad(lam1)
    lam2 = np.deg2rad(lam2)
    fi1 = np.deg2rad(fi1)
    fi2 = np.deg2rad(fi2)
    return abs(b**2*(lam2 - lam1)/2 *  (  ( np.sin(fi2)/(1 - e2*np.sin(fi2)**2) + (1/(2*e))*np.log((1 + e*np.sin(fi2)) / (1 - e*np.sin(fi2))) )  -  ( np.sin(fi1)/(1 - e2*np.sin(fi1)**2) + (1/(2*e))*np.log((1 + e*np.sin(fi1)) / (1 - e*np.sin(fi1))) )  ))

In [295]:
def pole_poligonu(lista_punktow):
    """Podajemy punkty kolejno. 'Rysujemy granice poligonu'. Działamy na płaszczyźnie."""
    return Polygon(lista_punktow).area

# /////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

## Testy

In [296]:
np.tan(np.deg2rad(45))

0.9999999999999999

In [297]:
A = [50.25, 20.75]
B = [50, 20.75]
C = [50.25, 21.25]
D = [50, 21.25]
sr_szer = [50.125, 21]
srodkowy = [50.1252704344052, 21.0006510711438]

In [298]:
to_gk(D[0], D[1], 19)

(5543273.892000523, 161308.28340695688)

In [299]:
to_1992(D[0], D[1])

(239393.6002761228, 661195.367608572)

In [300]:
to_2000(D[0], D[1])

(5540450.350009964, 7517922.548450623)

In [301]:
pole_czworokata(A[0],A[1],D[0],D[1])

994265196.0743111

In [302]:
pole_poligonu([A,B,D,C])

0.125

# /////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

## Zestawienie współrzędnych

In [303]:
main_df = pd.read_csv('dane.csv', sep=';')

In [304]:
main_df['Xgk'] = main_df.apply(lambda row: to_gk(row[0], row[1], 19)[0], axis=1)
main_df['Ygk'] = main_df.apply(lambda row: to_gk(row[0], row[1], 19)[1], axis=1)
main_df['X1992'] = main_df.apply(lambda row: to_1992(row[0], row[1])[0], axis=1)
main_df['Y1992'] = main_df.apply(lambda row: to_1992(row[0], row[1])[1], axis=1)
main_df['X2000'] = main_df.apply(lambda row: to_2000(row[0], row[1])[0], axis=1)
main_df['Y2000'] = main_df.apply(lambda row: to_2000(row[0], row[1])[1], axis=1)

In [305]:
main_df

,fi,lam,Xgk,Ygk,X1992,Y1992,X2000,Y2000
0,50.25000,20.750000,5.570121e+06,124812.227736,266221.512563,624724.859177,5.568256e+06,7.482171e+06
1,50.00000,20.750000,5.542315e+06,125464.200846,238435.404992,625376.375906,5.540450e+06,7.482077e+06
2,50.25000,21.250000,5.571078e+06,160469.906665,267178.205642,660357.577730,5.568256e+06,7.517829e+06
3,50.00000,21.250000,5.543274e+06,161308.283407,239393.600276,661195.367609,5.540450e+06,7.517923e+06
4,50.12500,21.000000,5.556667e+06,143014.239309,252777.110759,642914.129342,5.554323e+06,7.500000e+06
5,50.12527,21.000651,5.556698e+06,143059.985842,252808.414212,642959.843852,5.554353e+06,7.500047e+06


In [306]:
ad1_df = main_df.drop(['fi', 'lam'], axis=1)
ad1_df = ad1_df.style.format('{:.3f}')
ad1_df

,Xgk,Ygk,X1992,Y1992,X2000,Y2000
0,5570120.597,124812.228,266221.513,624724.859,5568256.030,7482170.562
1,5542315.026,125464.201,238435.405,625376.376,5540450.350,7482077.452
2,5571077.960,160469.907,267178.206,660357.578,5568256.030,7517829.438
3,5543273.892,161308.283,239393.600,661195.368,5540450.350,7517922.548
4,5556666.778,143014.239,252777.111,642914.129,5554323.110,7500000.000
5,5556698.103,143059.986,252808.414,642959.844,5554353.189,7500046.554


## Zestawienie pól powierzchni

In [307]:
A = [50.25, 20.75]
B = [50, 20.75]
C = [50.25, 21.25]
D = [50, 21.25]
sr_szer = [50.125, 21]
srodkowy = [50.1252704344052, 21.0006510711438]
AGK = to_gk(A[0], A[1], 19)
BGK = to_gk(B[0], B[1], 19)
CGK = to_gk(C[0], C[1], 19)
DGK = to_gk(D[0], D[1], 19)
A92 = to_1992(A[0], A[1])
B92 = to_1992(B[0], B[1])
C92 = to_1992(C[0], C[1])
D92 = to_1992(D[0], D[1])
A20 = to_2000(A[0], A[1])
B20 = to_2000(B[0], B[1])
C20 = to_2000(C[0], C[1])
D20 = to_2000(D[0], D[1])

In [308]:
pola_df = pd.DataFrame()
pola_df['P_elip'] = [pole_czworokata(A[0],A[1],D[0],D[1])]
pola_df['P_GK'] = [pole_poligonu([AGK, BGK, DGK, CGK])]
pola_df['P_1992'] = [pole_poligonu([A92, B92, D92, C92])]
pola_df['P_2000'] = [pole_poligonu([A20, B20, D20, C20])]

In [309]:
pola_df  # pola w m^2

,P_elip,P_GK,P_1992,P_2000
0,9.942652e+08,9.947608e+08,9.933686e+08,9.941083e+08


In [310]:
ad2_df = pola_df.style.format('{:.6f}')
ad2_df

,P_elip,P_GK,P_1992,P_2000
0,994265196.074311,994760761.493312,993368583.859984,994108281.708822
